# 基于瓜子二手车数据的二手电动车价格分析以及折价率分析模型

In [417]:
import numpy as np
import pandas as pd
import warnings
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import jn
from IPython.display import display, clear_output
import time

warnings.filterwarnings('ignore')
%matplotlib inline

## 模型预测的
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor

## 数据降维处理的
from sklearn.decomposition import PCA,FastICA,FactorAnalysis,SparsePCA

import lightgbm as lgb
import xgboost as xgb

## 参数搜索和评价的
from sklearn.model_selection import GridSearchCV,cross_val_score,StratifiedKFold,train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

## 数据读取

In [418]:
allData = pd.read_csv('../crawl_for_guazi/data.csv')
print(allData.shape)
allData.head()

(684, 14)


,car_name,car_brand,car_tag,price,new_price,complexOutlook,firstCert,odograph,allPower,carBelong,range,isDome,wheelBase,drivingMode
0,比亚迪 汉 2020款 EV 四驱高性能版旗舰型,比亚迪,汉,229000,303388,9成新,2020-08,1.9万公里,363kW,成都(川),550km,1,2920,双电机四驱
1,大众 ID.4 X 2021款 Pro 极智长续航版,大众,ID.4 X,189800,256049,9成新,2021-12,300公里,150kW,德州(鲁),555km,0,2765,后置后驱
2,宝马i3 2018款 豪华型(进口),宝马,宝马i3,138000,368842,8成新,2018-12,6.2万公里,125kW,上海(沪),271km,0,2570,后置后驱
3,特斯拉MODEL S 2014款 MODEL S 85,特斯拉,特斯拉MODEL S,285000,796735,7成新,2015-02,4.7万公里,270kW,烟台(鲁),502km,0,2960,后置后驱
4,江淮iEVA50 2018款 iEVA50 豪华型,江淮,江淮iEVA50,76000,199400,9成新,2019-01,100公里,85kW,合肥(皖),310km,1,2710,前置前驱


In [419]:
allData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 684 entries, 0 to 683
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   car_name        684 non-null    object
 1   car_brand       684 non-null    object
 2   car_tag         684 non-null    object
 3   price           684 non-null    int64 
 4   new_price       684 non-null    int64 
 5   complexOutlook  684 non-null    object
 6   firstCert       684 non-null    object
 7   odograph        684 non-null    object
 8   allPower        684 non-null    object
 9   carBelong       684 non-null    object
 10  range           684 non-null    object
 11  isDome          684 non-null    int64 
 12  wheelBase       684 non-null    int64 
 13  drivingMode     684 non-null    object
dtypes: int64(4), object(10)
memory usage: 74.9+ KB


In [420]:
allData.describe()

,price,new_price,isDome,wheelBase
count,684.000000,684.000000,684.000000,684.000000
mean,118067.084795,186476.032164,0.790936,2530.099415
std,91347.301067,125929.087693,0.406938,355.928549
min,14300.000000,0.000000,0.000000,1600.000000
25%,48950.000000,80107.000000,1.000000,2390.000000
50%,82650.000000,162711.500000,1.000000,2610.000000
75%,189050.000000,272368.750000,1.000000,2830.000000
max,648000.000000,905173.000000,1.000000,3110.000000


## 数据清洗

1. 该部分目的为将带单位的属性变为纯数字型属性，如将“7成新”变为“7”，以及时间属性，里程去单位
2. 具体的：
   1. 将成色属性去除尾部中文，并转换为float类型
   2. 将上牌年月属性转为时间单位，并增加一列“使用时间“，表示从首次上牌至今的天数
   3. 表显里程以纯数字形式显示，注意列表中带有”万“时需乘10000
   4. 车辆总功率去单位
   5. 续航里程去单位

### 1. 处理外观成色属性字段，将其转换为float类型

In [421]:
def dealOutlookStr(str):
    if type(str) is float:
        return str
    str = str.rstrip('成新')
    if len(str) > 1 and '.' not in str:
        str = float(str) / 10
    return str

allData['complexOutlook'] = allData['complexOutlook'].map(lambda x: np.float(dealOutlookStr(x)))
allData['complexOutlook'].describe()

count    684.000000
mean       8.646491
std        0.792813
min        6.000000
25%        8.000000
50%        9.000000
75%        9.000000
max        9.900000
Name: complexOutlook, dtype: float64

### 2. 将首次上牌修改为pandas.datetime形式，并新增一列表示从上牌至数据获取时(2022-01-26)的天数



In [422]:
allData['firstCert'] = pd.to_datetime(allData['firstCert'],format='%Y-%m')
allData['daysGone'] = ((pd.to_datetime('2022-01-26') - allData['firstCert']).dt.days)
allData['daysGone']

0       543
1        56
2      1152
3      2551
4      1121
       ... 
679    1578
680     117
681     178
682     239
683     970
Name: daysGone, Length: 684, dtype: int64

### 3. 处理表显里程数

In [423]:
def dealOdograph(str):
    if type(str) is float:
        return str
    str = str.rstrip('公里')
    if '万' in str:
        str = str.rstrip('万')
        str = float(str) * 10000
    return float(str)

allData['odograph'] = allData['odograph'].map(dealOdograph)
allData['odograph'].describe()

count       684.000000
mean      21844.444444
std       19784.169577
min         100.000000
25%        6000.000000
50%       17000.000000
75%       32250.000000
max      107000.000000
Name: odograph, dtype: float64

### 4. 处理车辆总功率

In [424]:
allData['allPower'] = allData['allPower'].map(lambda x: float(x.rstrip('kW')))

In [425]:
# 处理车辆续航里程
allData['range'].describe()
# temp = allData['range'].map(lambda x: 'km' in x)
# temp.unique()
def dealRange(str):
    if str is '-' or str is None:
        return None
    if type(str) is int:
        return str
    str = str.rstrip('km')
    return int(str)
allData['range'] = allData['range'].map(dealRange)

### 5. 删除new_price为0的行

In [426]:
allData = allData.drop(index=allData[allData['new_price'] == 0].index)

### 6. 添加损耗价格与损耗价格率
depreciation = new_price - price  
%depreciation = depriciation / new_price

In [427]:
def depri(df):
    return df['new_price'] - df['price'] if df['new_price'] - df['price'] > 0 else 0

def depri_per(df):
    return df['depri'] / df['new_price'] if df['new_price'] > 0 else 0

def preserve_per(df):
    return df['price'] / df['new_price'] if df['new_price'] > 0 else 0

allData['depri'] = allData.apply(depri, axis=1)
allData['depri_ratio'] = allData.apply(depri_per, axis=1)
allData['preserve_ratio'] = allData.apply(preserve_per, axis=1)

### 7. 清洗后的数据描述

In [428]:
allData.describe()

,price,new_price,complexOutlook,odograph,allPower,range,isDome,wheelBase,daysGone,depri,depri_ratio,preserve_ratio
count,681.000000,681.000000,681.000000,681.000000,681.000000,677.000000,681.000000,681.000000,681.000000,681.000000,681.000000,681.000000
mean,117956.073421,187297.512482,8.643612,21939.794420,115.016740,359.274742,0.790015,2528.073421,621.754772,69433.723935,0.347268,0.653156
std,91533.066291,125594.625687,0.792868,19775.342325,90.220023,145.652251,0.407597,355.396680,458.832451,66082.538336,0.211918,0.212764
min,14300.000000,31261.000000,6.000000,100.000000,15.000000,100.000000,0.000000,1600.000000,25.000000,0.000000,0.000000,0.069778
25%,48800.000000,80107.000000,8.000000,6000.000000,35.000000,270.000000,1.000000,2390.000000,270.000000,20307.000000,0.178162,0.537776
50%,82400.000000,164774.000000,9.000000,17000.000000,100.000000,353.000000,1.000000,2610.000000,482.000000,54793.000000,0.297215,0.702785
75%,189800.000000,273256.000000,9.000000,33000.000000,160.000000,468.000000,1.000000,2830.000000,970.000000,90840.000000,0.462224,0.821838
max,648000.000000,905173.000000,9.900000,107000.000000,577.000000,706.000000,1.000000,3110.000000,2551.000000,511735.000000,0.930222,1.177519


## 数据分析

### 1. 查看折价率为0的行  
车辆的二手价格甚至超过了新车的价格，这是不常见的

In [437]:
allData[allData['preserve_ratio'] > 1]

,car_name,car_brand,car_tag,price,new_price,complexOutlook,firstCert,odograph,allPower,carBelong,range,isDome,wheelBase,drivingMode,daysGone,depri,depri_ratio,preserve_ratio
524,特斯拉 Model Y 2022款 后轮驱动版,特斯拉,Model Y,385800,327638,9.9,2022-01-01,100.0,202.0,台州(浙),NaN,0,2890,后置后驱,25,0,0.0,1.177519
575,五菱汽车 宏光MINI EV 2020款 悦享款 三元锂,五菱,宏光MINI EV,46800,42116,9.5,2021-03-01,15000.0,20.0,上海(沪),170.0,1,1940,后置后驱,331,0,0.0,1.111217


In [441]:
allData['car_brand'].unique()

array(['比亚迪', '大众', '宝马', '特斯拉', '江淮', '吉利', '理想', '五菱', '北汽新能源', '宝骏',
       '传祺', '零跑汽车', '欧拉', '荣威', '天际汽车', '奇瑞', '知豆', '小鹏汽车', '广汽埃安',
       '威马汽车', '思铭', '广汽集团', '红旗', '北汽昌河', '别克', '蔚来', '启辰', '东风', '腾势',
       '丰田', '长安', '思皓', '理念', '奔驰', '东风风光', '江铃集团新能源', '海马', '开瑞', 'R汽车',
       'HYCAN合创', '哪吒汽车', '雷克萨斯', '雷诺', '东风风行', '日产', '众泰', '现代',
       'SERES赛力斯', '长安欧尚', '几何汽车', '猎豹', '奔腾', '新特汽车', '东南'], dtype=object)